In [16]:
import mlrun
import mlrun.feature_store as fstore
import queue

In [96]:
%%writefile serving.py

import mlrun
import mlrun.feature_store as fstore
import queue
import datetime
import json
import torch

def init_context(context):
    project = mlrun.get_or_create_project(name='stocks',user_project=True, context="./")
    tickers_data = {}
    start_time = datetime.datetime.now()-datetime.timedelta(7)
    end_time = datetime.datetime.now()-datetime.timedelta(0)
    train_dataset = fstore.get_offline_features('stocks', entity_timestamp_column='Datetime',  with_indexes=True, start_time=start_time, end_time=end_time)
    price_cols = ['Open','High','Low','Close']
    df = train_dataset.to_dataframe().reset_index(drop=False)
    df.fillna(value=1,inplace=True)
    normalized_df = df.copy()
    tickers = df['ticker'].unique()
    data = []
    labels = []
    normalized_df[price_cols] = (normalized_df[price_cols] - normalized_df[price_cols].mean()) / normalized_df[price_cols].std()
    normalized_df['Volume'] = (normalized_df['Volume'] - normalized_df['Volume'].mean()) / normalized_df['Volume'].std()
    for ticker in tickers:
        ticker_df = normalized_df[normalized_df['ticker'] == ticker].sort_values(by='Datetime').drop(['ticker','Datetime'],axis=1)
        q = queue.Queue()
        for idx,col in ticker_df[-5:].iterrows():
            q.put(col.values.tolist())
        tickers_data[ticker] = q
    setattr(context, 'tickers_data', tickers_data)
    
def handler(context,event):
    event.body = json.loads(event.body)
    context.logger.info(event)
    if(event.body['use_queue'] == True):
        print(torch.tensor([list(q.queue) for q in context.tickers_data.values()]).detach())
        return torch.tensor([list(q.queue) for q in context.tickers_data.values()]).detach()
    else:
        return torch.tensor(event.body['data']).detach

Writing serving.py


In [93]:
fn = mlrun.code_to_function("testing", kind='nuclio',image='mlrun/mlrun',requirements=['torch'])

In [94]:
fn.deploy()

> 2022-04-11 11:25:08,262 [info] Starting remote function deploy
2022-04-11 11:25:08  (info) Deploying function
2022-04-11 11:25:08  (info) Building
2022-04-11 11:25:08  (info) Staging files and preparing base images
2022-04-11 11:25:08  (warn) Python 3.6 runtime is deprecated and will soon not be supported. Please migrate your code and use Python 3.7 runtime (`python:3.7`) or higher
2022-04-11 11:25:08  (info) Building processor image
2022-04-11 11:27:23  (info) Build complete
2022-04-11 11:27:48  (info) Function deploy complete
> 2022-04-11 11:27:49,036 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-default-testing.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['default-testing-default.default-tenant.app.kjlirbtakxqf.iguazio-cd1.com/']}


'http://default-testing-default.default-tenant.app.kjlirbtakxqf.iguazio-cd1.com/'

In [95]:
import json
fn.invoke(body=json.dumps({'use_queue':True}),path='http://default-testing-default.default-tenant.app.kjlirbtakxqf.iguazio-cd1.com/')

> 2022-04-11 11:27:52,851 [info] invoking function: {'method': 'POST', 'path': 'http://default-testing-default.default-tenant.app.kjlirbtakxqf.iguazio-cd1.com/'}


b'tensor([[[-0.0255, -0.0249, -0.0241, -0.0231, -0.5897,  1.0000,  0.0000,\n           0.0000,  0.0000,  1.0000],\n         [-0.0226, -0.0243, -0.0238, -0.0255, -0.5665,  1.0000,  0.0000,\n           0.0000,  0.0000,  1.0000],\n         [-0.0258, -0.0261, -0.0262, -0.0285, -0.5461,  1.0000,  0.0000,\n           0.0000,  0.0000,  1.0000],\n         [-0.0284, -0.0288, -0.0291, -0.0269, -0.3820,  1.0000,  0.0000,\n           0.0000,  0.0000,  1.0000],\n         [-0.0282, -0.0304, -0.0262, -0.0285, -0.6249,  1.0000,  0.0000,\n           0.0000,  0.0000,  1.0000]],\n\n        [[ 1.1689,  1.1651,  1.1701,  1.1675, -0.5930,  0.0000,  0.0000,\n           1.0000,  0.0000,  1.0000],\n         [ 1.1684,  1.1647,  1.1624,  1.1587, -0.5658,  0.0000,  0.0000,\n           1.0000,  0.0000,  1.0000],\n         [ 1.1589,  1.1581,  1.1588,  1.1585, -0.5157,  0.0000,  0.0000,\n           1.0000,  0.0000,  1.0000],\n         [ 1.1588,  1.1595,  1.1595,  1.1624, -0.3196,  0.0000,  0.0000,\n           1.0000